<a href="https://colab.research.google.com/github/tigerak/1st_practice/blob/main/M12_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천 시스템 (Recommender Systems)

- 추천 시스템은 크게 두 가지로 구분 가능
 - 컨텐츠 기반 필터링 (content-based filtering)
 - 협업 필터링 (collaborative filtering)
- 두가지를 조합한 hybrid 방식도 가능
- 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
- 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

# Surprise

- 추천 시스템 개발을 위한 라이브러리
- 다양한 모델과 데이터 제공
- sklearn과 유사한 사용 방법

In [3]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 6.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617557 sha256=1b6dd5f9d0f1c118494b82a79f2869e2990633d90a5701f929ae56291501cc37
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


간단한 surprise 실습

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:
model = SVD()

cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9308  0.9468  0.9371  0.9273  0.9370  0.9358  0.0067  
MAE (testset)     0.7343  0.7477  0.7396  0.7312  0.7386  0.7383  0.0056  
Fit time          4.96    4.92    4.93    4.87    4.89    4.91    0.03    
Test time         0.26    0.17    0.17    0.19    0.26    0.21    0.04    


{'fit_time': (4.961913585662842,
  4.920889377593994,
  4.931941986083984,
  4.871452808380127,
  4.885516405105591),
 'test_mae': array([0.73430752, 0.74769709, 0.73958332, 0.73118646, 0.73855966]),
 'test_rmse': array([0.93075253, 0.94678284, 0.93713778, 0.92729031, 0.93704412]),
 'test_time': (0.26173973083496094,
  0.17145252227783203,
  0.1681210994720459,
  0.18688750267028809,
  0.2636239528656006)}

# 컨텐츠 기반 필터링 (Content-based Filtering)
- 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
 - 내가 지금까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
- 유사도를 기반으로 추천
- 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다
 - 장점
   - 많은 수의  사용자를 대상으로 쉽게 확장 가능
   - 사용자가 관심을 갖지 않던 상품 추천 가능
 - 단점
   - 입력 특성을 직접 설계해야하기 떄문에 않은 도데인 지식이 필요
   - 사용자의 기존 관심사항을 기반으로만 추천 가능

In [ ]:
import numpy as np
from surprise import Dataset

1. 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
- 나와 가장 유사도가 높은 사용자의 시청 목록을 추천

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [ ]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [ ]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [ ]:
# 인접행렬 만들기 : 1이 있는 위치가 데이터가 있는 위치
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 183, Best match ID : 275
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

2. 유클리드 거리를 사용해 추천
- 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 14.832396974191326, Best match ID : 737
[1 1 1 ... 0 0 0]
[1 1 0 ... 0 0 0]


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


3. 코사인 유사도를 사용해 추천

- 두 벡터가 이루고 있는 각을 계산

In [ ]:
def compute_cos_similarity(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 0.5278586163659506, Best match ID : 915
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 1]


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


- 기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 55.06359959174482, Best match ID : 737
[5 3 4 ... 0 0 0]
[5 3 0 ... 0 0 0]


In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 0.569065731527988, Best match ID : 915
[5 3 4 ... 0 0 0]
[4 3 3 ... 0 0 3]


# 협업 필터링 (Collaborative Filtering)

- 사용자와 항목의 유사성을 동시에 고려해 추천
- 기존에 내 관심사가 아닌 항목이라도 추천 가능
- 자동으로 임베딩 학습 가능
- 협업 필터링은 다음과 같은 장단점을 갖고 있다
 - 장점
   - 자동으로 임베딩을 학습하기 떄문에 도메인 지식이 필요 없다
   - 기존의 관심사가 아니더라도 추천 가능
 - 단점
   - 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
   - 추가 특성을 사용하기 어려움

In [ ]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)

KNN을 사용한 협업 필터링

In [ ]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9719  0.9884  0.9780  0.9722  0.9830  0.9787  0.0064  
MAE (testset)     0.7679  0.7808  0.7733  0.7655  0.7758  0.7726  0.0055  
Fit time          0.47    0.81    0.76    0.81    0.76    0.72    0.13    
Test time         6.62    8.70    8.76    6.70    4.16    6.99    1.69    


{'fit_time': (0.4690413475036621,
  0.8095924854278564,
  0.760784387588501,
  0.8113086223602295,
  0.7573864459991455),
 'test_mae': array([0.76787604, 0.78076494, 0.77334727, 0.76548065, 0.77576048]),
 'test_rmse': array([0.97190408, 0.98844527, 0.97803672, 0.97215301, 0.98295264]),
 'test_time': (6.616807460784912,
  8.697667598724365,
  8.756500720977783,
  6.704306602478027,
  4.155568599700928)}

In [ ]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9448  0.9340  0.9332  0.9383  0.9319  0.9365  0.0047  
MAE (testset)     0.7458  0.7369  0.7336  0.7415  0.7355  0.7386  0.0044  
Fit time          11.76   14.89   15.22   12.76   8.64    12.65   2.39    
Test time         0.61    0.36    0.43    0.27    0.16    0.37    0.15    


{'fit_time': (11.760620832443237,
  14.885550498962402,
  15.21551775932312,
  12.76208233833313,
  8.640636444091797),
 'test_mae': array([0.74575078, 0.7369383 , 0.73360762, 0.74146645, 0.73548639]),
 'test_rmse': array([0.94484045, 0.93403995, 0.93320115, 0.9383067 , 0.93193526]),
 'test_time': (0.605496883392334,
  0.35897374153137207,
  0.43065762519836426,
  0.27004480361938477,
  0.1643235683441162)}

In [ ]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9682  0.9579  0.9669  0.9623  0.9662  0.9643  0.0038  
MAE (testset)     0.7610  0.7525  0.7628  0.7563  0.7594  0.7584  0.0037  
Fit time          14.39   17.04   16.72   14.08   9.24    14.29   2.79    
Test time         0.69    0.47    0.43    0.25    0.15    0.40    0.19    


{'fit_time': (14.388464450836182,
  17.035849809646606,
  16.71802592277527,
  14.079005479812622,
  9.242494106292725),
 'test_mae': array([0.76102052, 0.75247118, 0.76284841, 0.75632701, 0.75936513]),
 'test_rmse': array([0.96816659, 0.95785551, 0.96690033, 0.96225934, 0.96619554]),
 'test_time': (0.6906940937042236,
  0.47449636459350586,
  0.42613744735717773,
  0.25169849395751953,
  0.1514418125152588)}

In [ ]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9214  0.9176  0.9182  0.9219  0.9183  0.9195  0.0018  
MAE (testset)     0.7194  0.7171  0.7208  0.7255  0.7213  0.7208  0.0028  
Fit time          676.42  676.33  666.78  675.51  170.17  573.04  201.47  
Test time         12.77   12.05   12.16   6.90    3.20    9.42    3.76    


{'fit_time': (676.4161787033081,
  676.3269464969635,
  666.7840266227722,
  675.5080344676971,
  170.17452549934387),
 'test_mae': array([0.71941384, 0.71706722, 0.72084548, 0.72551692, 0.72130241]),
 'test_rmse': array([0.92142586, 0.91760615, 0.91816809, 0.92187449, 0.91826335]),
 'test_time': (12.76694107055664,
  12.049471378326416,
  12.157771348953247,
  6.902799129486084,
  3.199129104614258)}

# 하이브리드 (Hybrid)

- 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
- 많은 하이브리드 방식이 존재
- 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [4]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [5]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
print(shape)

adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating
adj_matrix

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
(943, 1682)


array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [13]:
U, S, V = randomized_svd(adj_matrix, n_components=2) # 사용자, 특이값 벡터, 항목(아이템의 행렬)
S = np.diag(S)

print(U.shape)
print(S.shape)
print(V.shape, '\n')
print(U, '\n')
print(S, '\n')
print(V, '\n')

(943, 2)
(2, 2)
(2, 1682) 

[[ 0.06580431  0.00597507]
 [ 0.01402104 -0.04662601]
 [ 0.00565798 -0.02561846]
 ...
 [ 0.00744452 -0.02502129]
 [ 0.02403119  0.0080961 ]
 [ 0.04224209 -0.01092715]] 

[[640.63362257   0.        ]
 [  0.         244.83634567]] 

[[ 9.59509371e-02  3.51795155e-02  1.99288117e-02 ...  3.03747116e-05
   3.31055915e-04  3.16852950e-04]
 [-8.72398147e-02 -7.02507073e-03 -2.86182142e-02 ... -4.48134999e-04
   1.05230265e-04  2.03150753e-04]] 



In [9]:
np.matmul(np.matmul(U, S), V) # 복원

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

1. 사용자 기반 추천
- 나와 비슷한 취향을 가진 다른 사용자들의 행동을 추천
- 사용자 특징 벡터의 유사도 사용

In [15]:
def compute_cos_similarity(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [17]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 0.9999942295956324, Best match ID : 235
[0.06580431 0.00597507]
[0.03467744 0.00326754]


In [19]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


2. 항목 기반 추천
- 내가 본 항목과 비슷한 항복을 추천
- 항목 특징 벡터의 유사도 사용

In [29]:
my_id, my_vector = 0, V.T[0] # V값은 곱으로 Decomposition 했기 때문
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 0.9999999951364144, Best match ID : 1287
[ 0.09595094 -0.08723981]
[ 0.00193695 -0.00176145]


In [32]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

- 비 음수 행렬 분해를 사용한 하이브리드 추천

In [33]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [34]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

np.matmul(A, B)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


array([[3.71103931e+00, 1.48480986e+00, 7.39556918e-01, ...,
        3.64529861e-03, 1.45531613e-02, 1.44129285e-02],
       [2.11742679e+00, 2.36669558e-01, 5.51605522e-01, ...,
        4.76780705e-03, 2.37805973e-05, 0.00000000e+00],
       [9.85455002e-01, 1.10146524e-01, 2.56718402e-01, ...,
        2.21894770e-03, 1.10675414e-05, 0.00000000e+00],
       ...,
       [1.04460235e+00, 1.16757556e-01, 2.72126728e-01, ...,
        2.35212970e-03, 1.17318190e-05, 0.00000000e+00],
       [1.45795166e+00, 5.42097365e-01, 2.99304935e-01, ...,
        1.61367845e-03, 5.15821593e-03, 5.10692111e-03],
       [2.44652929e+00, 9.41451647e-01, 4.95503558e-01, ...,
        2.56793344e-03, 9.08679105e-03, 8.99778817e-03]])

1. 사용자 기반 추천

In [35]:
def compute_cos_similarity(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [36]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 0.9999942295956324, Best match ID : 235
[0.06580431 0.00597507]
[0.03467744 0.00326754]


In [37]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


2. 항목 기반 추천

In [38]:
my_id, my_vector = 0, V.T[0] # V값은 곱으로 Decomposition 했기 때문
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best match : {}, Best match ID : {}'.format(best_match, best_match_id))
print(my_vector)
print(best_match_vector)

Best match : 0.9999999951364144, Best match ID : 1287
[ 0.09595094 -0.08723981]
[ 0.00193695 -0.00176145]


In [39]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 